In [ ]:
import tensorflow as tf
import argparse
import os.path
from cifar100_models import *
from cifar100_input import *
from utils import *
import itertools
import logging
import pickle

tf.logging.set_verbosity(tf.logging.INFO)

# get TF logger
log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# create file handler which logs even debug messages
fh = logging.FileHandler('tensorflow_cifar100.log')
fh.setLevel(logging.INFO)
fh.setFormatter(formatter)
log.addHandler(fh)

#### SET THIS BEFORE RUNNING
DIR = "cifar100"
NAME = "cifar100_scm_{}_{}_{}_{}"
PATH = os.path.join(DIR,NAME)

def train(model, name, restore_point = None, wd1=0., wd2=0., wd3=0., drop1=1., save=True):
    with tf.Session() as sess:
        model._restore_or_init(sess, restore_point)
        model.weight_decay_layer_1.assign(wd1)
        model.weight_decay_layer_2.assign(wd2)
        model.weight_decay_layer_3.assign(wd3)
        model.drop_fc1.assign(drop1)
        
        try:
            while True:
                sess.run(model.train_step)
        except tf.errors.OutOfRangeError as e:
            pass
        
        global_step = model.global_step.eval()
        if save:
            checkpoint_name = CHECKPOINT.format(name) + "_" + str(global_step).rjust(10,"0")
            tf.logging.info("Saving to: " + checkpoint_name)
            model.saver.save(sess, checkpoint_name)
    return checkpoint_name
        
def evaluate(model, restore_point = None):
    acc = []
    with tf.Session() as sess:
        model._restore_or_init(sess, restore_point)
        try:
            while True:
                acc.append(model.accuracy.eval())
        except tf.errors.OutOfRangeError as e:
            pass
    return np.mean(acc)

@np.vectorize
def nplte(x,y):
    return x<=y

def train_to_convergence(train_files, test_files, wd1=0., wd2=0., wd3=0., drop1=1., save_rate=10):
    acc = []
    name = PATH.format(wd1,wd2,wd3,drop1)
    make_dirs(CHECKPOINT.format(name))
    checkpoint = latest_checkpoint(name)
    i = 0
    while len(acc)<30 or nplte(np.array(acc[-20:]) - np.array(acc[-21:-1]),0).sum() < 13: # until the accuracy is just as likely to go down as up
        tf.reset_default_graph()
        train_data = Cifar100Record(train_files, epochs=1)
        model = Cifar100ShallowConvolutionalModel(train_data.image, train_data.label)
        acc.append(evaluate(model, restore_point = checkpoint))
        tf.reset_default_graph()
        train_data = Cifar100Record(train_files, epochs=1)
        model = Cifar100ShallowConvolutionalModel(train_data.image, train_data.label)
        save = True
        checkpoint = train(model, name, restore_point = checkpoint, wd1=wd1, wd2=wd2, wd3=wd3, drop1=drop1, save=save)
        tf.logging.info(acc[-21:])
        i+=1
    return acc
        
def train_a_bunch_of_cifar100s():
    wd1_space = np.linspace(0.,0.012,3) # they use 0.004 in the tensorflow example
    wd2_space = np.linspace(0.,0.012,3)
    wd3_space = np.linspace(0.,0.012,3)
    drop1_space = np.linspace(0.2,1,3)
    wd_drop_grid = itertools.product(wd1_space, wd2_space, wd3_space, drop1_space)
    train_files, test_files, validation_files = maybe_download_and_extract()
    try:
        with open("completed_cifar100.p",'rb') as f:
            completed = pickle.load(f)
    except:
        completed = []
    i = 0
    for wd1,wd2,wd3,drop1 in wd_drop_grid:
        if i < len(completed): #skip values until we get to a new one
            i+=1
            continue
        acc = train_to_convergence(train_files,test_files,wd1,wd2,wd3,drop1)
        tf.logging.info(acc[-10:])
        completed.append((wd1,wd2,wd3,drop1,acc))
        i+=1
        with open("completed_cifar100.p","wb") as f:
            pickle.dump(completed,f)

In [ ]:
make_dirs(ROOT.format(DIR))
train_a_bunch_of_cifar100s()

INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000118400
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000118400
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000118800
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000118800 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.76797503]
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000118800
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000118800
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000119200
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000119200 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.767975

INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000124000
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000124400
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000124400 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.76797503, 0.75967497, 0.77365005, 0.76924998, 0.77607501, 0.77662504, 0.77142501, 0.77622503, 0.77612501, 0.77765, 0.77562499, 0.78455001, 0.78702497, 0.78860003, 0.79087502]
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000124400
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000124400
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000124800
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000124800 is not in all_model_checkpoint_p

INFO:tensorflow:[0.77662504, 0.77142501, 0.77622503, 0.77612501, 0.77765, 0.77562499, 0.78455001, 0.78702497, 0.78860003, 0.79087502, 0.78885001, 0.80074996, 0.80042505, 0.80002505, 0.80052507, 0.80645007, 0.79197496, 0.81067502, 0.80972505, 0.81040001, 0.80794996]
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000128800
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000128800
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000129200
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000129200 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.77142501, 0.77622503, 0.77612501, 0.77765, 0.77562499, 0.78455001, 0.78702497, 0.78860003, 0.79087502, 0.78885001, 0.80074996, 0.80042505, 0.80002505, 0.80052507, 0.80645007, 0.79197496, 0.81067502, 0.80972505, 0.81040001, 0.80794996, 0.81569

INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000133200
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000133200
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000133600
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000133600 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.80042505, 0.80002505, 0.80052507, 0.80645007, 0.79197496, 0.81067502, 0.80972505, 0.81040001, 0.80794996, 0.81569999, 0.81932497, 0.81972504, 0.81997496, 0.82342499, 0.82779998, 0.83372504, 0.827025, 0.83122504, 0.83645004, 0.83755004, 0.83832496]
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000133600
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000133600
INFO:tensorflow:Saving to: ../mo

INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000137600
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000138000
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000138000 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.81972504, 0.81997496, 0.82342499, 0.82779998, 0.83372504, 0.827025, 0.83122504, 0.83645004, 0.83755004, 0.83832496, 0.83922499, 0.84039998, 0.83767503, 0.8445251, 0.84949994, 0.84517503, 0.84812498, 0.84687501, 0.84690005, 0.85967499, 0.86339998]
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000138000
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000138000
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.2/model.ckpt_0000138400
INFO:tensorflow:../models/cifar100/cifar100_scm_

...

INFO:tensorflow:[0.99550003, 0.99649996, 0.9964, 0.99615008, 0.99662501, 0.9971, 0.99689996, 0.99729997, 0.99729997, 0.99747497, 0.99769998, 0.99792498, 0.99777496, 0.99799997, 0.99790001, 0.99827498, 0.99809998, 0.99817497, 0.99829996, 0.99854994, 0.99837494]
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000204400
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000204400
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000204800
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000204800 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.99649996, 0.9964, 0.99615008, 0.99662501, 0.9971, 0.99689996, 0.99729997, 0.99729997, 0.99747497, 0.99769998, 0.99792498, 0.99777496, 0.99799997, 0.99790001, 0.99827498, 0.99809998, 0.998

INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000208400
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000208400
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000208800
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000208800 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.99792498, 0.99777496, 0.99799997, 0.99790001, 0.99827498, 0.99809998, 0.99817497, 0.99829996, 0.99854994, 0.99837494, 0.99860001, 0.99859995, 0.99862498, 0.9985249, 0.99862504, 0.99877495, 0.99877501, 0.9989, 0.99897492, 0.9989, 0.99899995]
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000208800
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_

INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000212800 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.99860001, 0.99859995, 0.99862498, 0.9985249, 0.99862504, 0.99877495, 0.99877501, 0.9989, 0.99897492, 0.9989, 0.99899995, 0.99902499, 0.99902499, 0.99910003, 0.99899995, 0.99910003, 0.99912506, 0.99914992, 0.99917495, 0.99919999, 0.99927503]
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000212800
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000212800
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000213200
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000213200 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.99859995, 0.99862498, 0.998

INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000216800
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000216800
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000217200
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000217200 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.99902499, 0.99902499, 0.99910003, 0.99899995, 0.99910003, 0.99912506, 0.99914992, 0.99917495, 0.99919999, 0.99927503, 0.99919999, 0.99932498, 0.99927497, 0.99922502, 0.99934995, 0.99937499, 0.99940002, 0.99940002, 0.99940002, 0.99942505, 0.99949998]
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000217200
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100

INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000221200
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000221200 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.99919999, 0.99932498, 0.99927497, 0.99922502, 0.99934995, 0.99937499, 0.99940002, 0.99940002, 0.99940002, 0.99942505, 0.99949998, 0.99952501, 0.99947494, 0.99952501, 0.99955004, 0.99955004, 0.99967498, 0.99964994, 0.99957496, 0.99967498, 0.99967498]
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000221200
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000221200
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/model.ckpt_0000221600
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.006_0.6000000000000001/mod